In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import time
from bs4 import BeautifulSoup
import re
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, NoSuchFrameException

In [53]:
import pandas as pd
import numpy as np

df = pd.DataFrame(columns=['cafe_name', "cafe_type", 'starring', 'visitor_review', 'blog_review', 'address', 'business_hours', 'phone_number', 'connection_site', 'description',
"menu", "keyword", 'keyword_number'])

In [3]:
cafe_name_list = ['우주라이크커피 경산점',
 '하늘아래사진관',
 '말그미옥상',
 '호미화방',
 '코너커피',
 '줄라이모닝',
 '카페로커피',
 '엠프티커피',
 '다올',
 '텀블루']

In [70]:
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException

def get_new_reference(driver, xpath):
    """
    페이지나 프레임 전환 후 요소를 다시 찾아서 반환합니다.
    """
    try:
        return WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
        print("Timeout while waiting for element.")
        return None

In [59]:
def cafe_crawling(driver):
#     try:
#         WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, "place_section no_margin OP4V8")))
#         time.sleep(1)
#     except:
#         driver = switch_driver(driver, "searchIframe")
#         print(1)
#         return None
    time.sleep(1.5)
    try:
        description_a_link = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div/div[2]/div/div/div[7]/div/a/span[1]')
        description_a_link.click()
    except Exception as ex:
        pass

    # 기본 카페 정보 크롤링
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    if not "대구" in address and not "경산" in address:
        return None

    cafe_name = soup.find('span', class_='Fc1rA').text
    cafe_type = soup.find('span', class_='DJJvD').text
    reviews_div = soup.find('div', class_='dAsGb')
    starring = float(reviews_div.find('span', class_='PXMot LXIwF').text.strip()[2:]) if reviews_div.find('span', class_='PXMot LXIwF') else 0
    
    if not reviews_div.find_all('span', class_='PXMot') or len(reviews_div.find_all('span', class_='PXMot')) < 3:
        return None
    
    visitor_reviews = int(reviews_div.find_all('span', class_='PXMot')[1 if starring else 0].text.strip().split(" ")[1].replace(",", ""))
    blog_reviews = int(reviews_div.find_all('span', class_='PXMot')[2 if starring else 1].text.strip().split(" ")[1].replace(",", ""))
    
    if visitor_reviews < 10:
        return None
    
    description = soup.find_all('span', class_='zPfVt')[1].text if len(soup.find_all('span', class_='zPfVt')) > 1 else "" 
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    business_hours = soup.find('time').text if soup.find('time') else "" 
    phone_number = soup.find('span', class_='xlx7Q').text if soup.find('span', class_='xlx7Q') else ""

    # 키워드로 넘어가기
    keyword_tab_link = driver.find_element(By.XPATH, "//div[@class='flicking-camera']/a[.//span[contains(text(), '리뷰')]]")
    keyword_tab_link.click()
    keyword_list_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Tvx37")))
    keyword_list_link.click()

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    keyword_number = int(soup.find('em').text)

    keywords = {}
    ui_tags = soup.find('ui', class_='uNsI9')
    li_tags = soup.find_all('li', class_='nbD78')

    for li in li_tags:
        # 첫 번째 숫자를 int 형으로 변환
        text = li.find('span', class_='nWiXa').text.strip('"')
        numbers = re.findall(r'\d+', li.find('span', class_='TwM9q').text)
        number = int(numbers[0]) if numbers else None
        keywords[text] = number
    
    return [cafe_name, cafe_type, starring, visitor_reviews, blog_reviews, address, business_hours, phone_number, "", description, "", keywords, keyword_number]

In [60]:
def switch_driver(driver, iframe):
    try:
        driver.switch_to.default_content()
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, iframe)))
    except NoSuchFrameException:
        print(f"Frame '{iframe}' not found.")
    return driver

In [61]:
# def switch_iframe(driver):
#     try:
#         # 현재 iframe 확인
#         current_iframe = driver.find_element(By.TAG_NAME, 'iframe').get_attribute('id')

#         # 현재 iframe이 'searchIframe'인 경우 'entryIframe'으로 전환
#         if current_iframe == 'searchIframe':
#             driver.switch_to.default_content() # 메인 컨텐츠로 돌아감
#             driver.switch_to.frame('entryIframe') # 'entryIframe'으로 전환

#         # 현재 iframe이 'entryIframe'인 경우 'searchIframe'으로 전환
#         elif current_iframe == 'entryIframe':
#             driver.switch_to.default_content() # 메인 컨텐츠로 돌아감
#             driver.switch_to.frame('searchIframe') # 'searchIframe'으로 전환

#     except NoSuchElementException:
#         # iframe을 찾을 수 없는 경우, 'searchIframe'으로 전환 시도
#         try:
#             driver.switch_to.default_content() # 메인 컨텐츠로 돌아감
#             driver.switch_to.frame('searchIframe') # 'searchIframe'으로 전환
#         except NoSuchElementException:
#             # 'searchIframe'도 찾을 수 없는 경우
#             print("searchIframe을 찾을 수 없습니다.")

In [71]:
# Selenium을 사용하여 웹 드라이버 초기화
driver = webdriver.Chrome()

#... 이전 코드 ...

for cafe_name in cafe_name_list:
    # 검색
    driver.get("https://map.naver.com/p/search/" + cafe_name)
    time.sleep(1.5)  # 추가 대기 시간
    
    current_url = driver.current_url
    url_point = current_url.split("/")

    if len(url_point) <= 6:
        driver = switch_driver(driver, "searchIframe")
        scrollable_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container")))

        for _ in range(5):
            driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
            time.sleep(1)  # 적절한 로딩 시간 기다림

        li_elements = scrollable_div.find_elements(By.TAG_NAME, "li") 

        for idx in range(len(li_elements)):
            try:
                # 페이지나 프레임 전환 후 요소를 다시 찾습니다.
                scrollable_div = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container")))
                cafe_link = scrollable_div.find_element(By.XPATH, f'./ul/li[{idx+1}]/div[1]/div[2]/a[1]')
                cafe_link.click()

                driver = switch_driver(driver, "entryIframe")

                cafe_row = cafe_crawling(driver)
                if cafe_row:
                    df.loc[len(df)] = cafe_row
                    print(cafe_row)

                driver = switch_driver(driver, "searchIframe")
            except StaleElementReferenceException:
                # 요소가 더 이상 유효하지 않을 경우 다시 시도
                continue
    else:   
        driver = switch_driver(driver, "entryIframe")
        cafe_row = cafe_crawling(driver)
        if cafe_row:
            df.loc[len(df)] = cafe_row
            print(cafe_row)


['우주라이크커피 경산점', '카페', 4.77, 84, 4, '경북 경산시 대학로10길 24 1층', '21:00에 영업 종료', '053-710-8852', '', '반갑습니다 : ) 우주라이크 커피 경산점 인사드립\n니다! 새롭고, 신선하고, 합리적인 커피! 브라질 내\n추럴, 콜롬비아 수프리모 워시드, 에티오피...', '', {'커피가 맛있어요': 34, '친절해요': 23, '디저트가 맛있어요': 18, '음료가 맛있어요': 15, '가성비가 좋아요': 12, '특별한 메뉴가 있어요': 9, '매장이 청결해요': 9, '대화하기 좋아요': 7, '주차하기 편해요': 6, '좌석이 편해요': 5}, 61]
['말그미옥상', '케이크전문', 4.55, 91, 153, '대구 남구 계명중앙1길 25 2층', '20:00에 영업 종료', '0507-1310-3966', '', "'맑음' 이라는 말에서 따온 순 우리말\n말그미라는 단어를 사용한 '말그미옥상' 입니다.", '', {'커피가 맛있어요': 35, '디저트가 맛있어요': 17, '대화하기 좋아요': 17, '음료가 맛있어요': 16, '친절해요': 15, '인테리어가 멋져요': 10, '매장이 청결해요': 10, '뷰가 좋아요': 10, '특별한 메뉴가 있어요': 6, '사진이 잘 나와요': 4}, 69]
Stale element reference. Attempting to find element again.
Timeout while waiting for element.
Stale element reference. Attempting to find element again.
Timeout while waiting for element.
Timeout while waiting for element.
Timeout while waiting for element.
Timeout while waiting for element.
Timeout while waiting for element.
Timeout w

In [58]:
df

,cafe_name,cafe_type,starring,visitor_review,blog_review,address,business_hours,phone_number,connection_site,description,menu,keyword,keyword_number
0,아눅 월성,"카페,디저트",0.00,623,153,대구 달서구 조암남로32길 16 1층,08:00에 영업 시작,0507-1485-0604,,We exist so that you may rest assured and eat ...,,"{'커피가 맛있어요': 481, '디저트가 맛있어요': 420, '인테리어가 멋져요...",615
1,웜 브리즈,"카페,디저트",0.00,144,94,대구 달성군 옥포읍 명곡로 579 1층,11:00에 영업 시작,0507-1449-5577,,잔잔하고 따뜻한 분위기를 추구하는 카페 웜브리즈입니다.\n\n송해공원에서 차로 3분...,,"{'인테리어가 멋져요': 80, '커피가 맛있어요': 78, '친절해요': 61, ...",142
2,무화커피,"카페,디저트",4.54,1373,1093,대구 수성구 국채보상로186길 81 2층 무화커피,10:30에 영업 시작,0507-1437-9188,,안녕하세요 범어동 법원 뒤 야시골 공원길의\n작은 로스터리 카페 **무화커피** 입...,,"{'커피가 맛있어요': 595, '뷰가 좋아요': 401, '디저트가 맛있어요': ...",867
3,굿테이스트커피 by 커피맛을조금아는남자,"카페,디저트",0.00,766,474,대구 동구 팔공산로277길 26 26-1(백안동),10:00에 영업 시작,070-8865-4603,,2007년 봉덕동에서 시작한 커피맛을 조금 아는남자는\n현재 범어 본점을 기점으로 ...,,"{'커피가 맛있어요': 462, '뷰가 좋아요': 344, '주차하기 편해요': 2...",739
4,카페댐댐,"카페,디저트",0.00,778,274,대구 군위군 삼국유사면 삼국유사로 438-16,10:00에 영업 시작,0507-1486-3039,,카페댐댐은 군위댐의 아름다운 경치를 한눈에 볼 수 있으며 스페셜티 원두를 이용한 고...,,"{'뷰가 좋아요': 487, '커피가 맛있어요': 341, '디저트가 맛있어요': ...",736
5,부루잉커피바,카페,0.00,61,56,대구 중구 명륜로23길 83 1층,14:00에 영업 시작,0508-779-3664,,부루잉Brewing으로 에스프레소바처럼 독특한 음료를 만드는 곳\n나아갈 부 / 정...,,"{'커피가 맛있어요': 45, '친절해요': 36, '특별한 메뉴가 있어요': 26...",61
6,무화커피,"카페,디저트",4.54,1373,1093,대구 수성구 국채보상로186길 81 2층 무화커피,10:30에 영업 시작,0507-1437-9188,,안녕하세요 범어동 법원 뒤 야시골 공원길의\n작은 로스터리 카페 **무화커피** 입...,,"{'커피가 맛있어요': 595, '뷰가 좋아요': 401, '디저트가 맛있어요': ...",867
7,굿테이스트커피 by 커피맛을조금아는남자,"카페,디저트",0.00,766,474,대구 동구 팔공산로277길 26 26-1(백안동),10:00에 영업 시작,070-8865-4603,,2007년 봉덕동에서 시작한 커피맛을 조금 아는남자는\n현재 범어 본점을 기점으로 ...,,"{'커피가 맛있어요': 462, '뷰가 좋아요': 344, '주차하기 편해요': 2...",739
8,룰리커피 가창점,"카페,디저트",0.00,1270,1436,대구 달성군 가창면 우록길 76,10:00에 영업 시작,0507-1382-7637,,,,"{'커피가 맛있어요': 819, '뷰가 좋아요': 426, '인테리어가 멋져요': ...",1242
9,룰리커피,"카페,디저트",4.49,2331,2271,대구 수성구 고모로 188,10:00에 영업 시작,0507-1397-4669,,좋은 커피를 보내드립니다.,,"{'커피가 맛있어요': 969, '주차하기 편해요': 352, '디저트가 맛있어요'...",1437


In [57]:
# WebDriver 초기화
driver = webdriver.Chrome() # Chrome 드라이버 사용
driver.get("https://map.naver.com/p/search/대구 커피") # 네이버 지도 접속

# 결과 로드 대기
time.sleep(3) # 페이지 로드를 위해 잠시 대기

# searchIframe으로 컨텍스트 전환 + div scroll 내리기
driver.switch_to.frame("searchIframe")
scroll_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container")))
for _ in range(5):
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scroll_div)
    time.sleep(1)

# 목록을 담고 있는 ul 태그를 찾고, 그 안의 모든 li 태그를 가져옴
cafe_list = scroll_div.find_elements(By.TAG_NAME, "li")
cafe_size = len(cafe_list)
idx = 0

# while idx < cafe_size:
    # cafe_list = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container"))).find_elements(By.TAG_NAME, "li")
    # 각 li 태그를 순회하면서 카페이름에 해당하는 a 링크를 클릭
for item in cafe_list:
    try:
        place_bluelink = item.find_element(By.CLASS_NAME, "place_bluelink")
        # 부모의 부모인 a 태그 찾기 및 클릭
        parent_a_tag = place_bluelink.find_element(By.XPATH, "../..")
        parent_a_tag.click()
        # cafe_link = item.find_element(By.CLASS_NAME, "P7gyV")
        # cafe_link.click()
        time.sleep(3)

        # 필요한 작업 수행 (예: 상세 정보 수집)
        driver.switch_to.default_content()
        driver.switch_to.frame("entryIframe")
        # print(cafe_crawling2())
        cafe_row = cafe_crawling2()
        if cafe_row:
            df.loc[len(df)] = cafe_row
            print(cafe_row)

        # 다음 카페로 이동하기 전에 searchIframe으로 다시 컨텍스트 전환
        driver.switch_to.default_content()
        driver.switch_to.frame("searchIframe")
        
        # idx += 1
    except Exception as e:
        print(e)
        # # print("새로고침")
        # driver.refresh()
        # # cafe_list = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container"))).find_elements(By.TAG_NAME, "li")
        # driver.switch_to.frame("searchIframe")
        # scroll_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container")))
        # for _ in range(5):
        #     driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scroll_div)
        #     time.sleep(1)
        pass


# 원래 프레임으로 돌아감
driver.switch_to.default_content()


['카페댐댐', '카페,디저트', 0, 778, 274, '대구 군위군 삼국유사면 삼국유사로 438-16', '10:00에 영업 시작', '0507-1486-3039', '', '카페댐댐은 군위댐의 아름다운 경치를 한눈에 볼 수 있으며 스페셜티 원두를 이용한 고급진 커피를 제공하고 있습니다. 베이커리 또한 다양하게 준비되어 있으니 카페댐댐에 오셔서 물멍 때리다 가세요:)', '', {'뷰가 좋아요': 487, '커피가 맛있어요': 341, '디저트가 맛있어요': 255, '인테리어가 멋져요': 229, '사진이 잘 나와요': 226, '음료가 맛있어요': 184, '매장이 청결해요': 174, '친절해요': 160, '좌석이 편해요': 102, '대화하기 좋아요': 87, '주차하기 편해요': 79, '화장실이 깨끗해요': 61, '특별한 메뉴가 있어요': 59, '가성비가 좋아요': 22, '집중하기 좋아요': 18, '매장이 넓어요': 5, '야외 공간이 멋져요': 1, '반려동물과 가기 좋아요': 1}, 736]
['부루잉커피바', '카페', 0, 61, 56, '대구 중구 명륜로23길 83 1층', '14:00에 영업 시작', '0508-779-3664', '', '부루잉Brewing으로 에스프레소바처럼 독특한 음료를 만드는 곳\n나아갈 부 / 정성스러울 루 / 땅이름 잉\n정성으로 고객에게 나아는 공간이란 뜻의 소명커피바 2번째 컨셉매장', '', {'커피가 맛있어요': 45, '친절해요': 36, '특별한 메뉴가 있어요': 26, '음료가 맛있어요': 25, '디저트가 맛있어요': 21, '매장이 청결해요': 17, '인테리어가 멋져요': 15, '대화하기 좋아요': 11, '사진이 잘 나와요': 8, '집중하기 좋아요': 8, '가성비가 좋아요': 3, '컨셉이 독특해요': 1, '코스요리가 알차요': 1, '혼술하기 좋아요': 1, '화장실이 깨끗해요': 1}, 61]
['무화커피', '카페,디저트', 4.54, 1373, 1093, '대구 수성

In [56]:
def cafe_crawling2():
    description_state = False
    try:
        description_a_link = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > a > span.zPfVt")
        description_a_link.click()
        description_state = True
    except Exception as ex:
        pass

    # 기본 카페 정보 크롤링
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    if not "대구" in address and not "경산" in address:
        return None

    cafe_name = soup.find('span', class_='Fc1rA').text
    cafe_type = soup.find('span', class_='DJJvD').text
    
    # reviews_div = soup.find('div', class_='dAsGb')
    # starring = float(reviews_div.find('span', class_='PXMot LXIwF').text.strip()[2:]) if reviews_div.find('span', class_='PXMot LXIwF') else 0
    # if not reviews_div.find_all('span', class_='PXMot') or len(reviews_div.find_all('span', class_='PXMot')) < 3:
    #     return None
    # visitor_reviews = int(reviews_div.find_all('span', class_='PXMot')[1 if starring else 0].text.strip().split(" ")[1].replace(",", ""))
    # blog_reviews = int(reviews_div.find_all('span', class_='PXMot')[2 if starring else 1].text.strip().split(" ")[1].replace(",", ""))
    
    try:
        # 별점 요소 찾기
        star_rating_element = driver.find_element(By.CSS_SELECTOR, "div.dAsGb > span.PXMot.LXIwF")
        starring = float(star_rating_element.text.strip()[2:])
    except NoSuchElementException:
        # 별점 요소가 없는 경우
        starring = 0

    try:
        # 방문자 리뷰 요소 찾기
        visitor_review_element = driver.find_element(By.XPATH, "//span[@class='PXMot']/a[contains(@href, '/review/visitor')]")
        visitor_reviews = int(visitor_review_element.text.strip().split(" ")[1].replace(",", ""))
    except NoSuchElementException:
        # 방문자 리뷰 요소가 없는 경우
        visitor_reviews = 0
        
    try:
        # 블로그 리뷰 요소 찾기
        blog_review_element = driver.find_element(By.XPATH, "//span[@class='PXMot']/a[contains(@href, '/review/ugc')]")
        blog_reviews = int(blog_review_element.text.strip().split(" ")[1].replace(",", ""))
    except NoSuchElementException:
        # 블로그 리뷰 요소가 없는 경우
        blog_reviews = 0
    
    if visitor_reviews < 10:
        return None
    #PIbes > O8qbU > vV_z_ > zPfVt
    # description = soup.find_all('span', class_='zPfVt')[1].text if len(soup.find_all('span', class_='zPfVt')) > 1 else ""
    if description_state:
        description = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > a > span.zPfVt").text
    else:
        try:
            description = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > div.xHaT3 >  span.zPfVt").text
        except:
            description = ""
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    business_hours = soup.find('time').text if soup.find('time') else "" 
    phone_number = soup.find('span', class_='xlx7Q').text if soup.find('span', class_='xlx7Q') else ""

    # 키워드로 넘어가기
    keyword_tab_link = driver.find_element(By.XPATH, "//div[@class='flicking-camera']/a[.//span[contains(text(), '리뷰')]]")
    keyword_tab_link.click()
    try:
        keyword_list_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Tvx37")))
        keyword_list_link.click()
    except:
        return [cafe_name, cafe_type, starring, visitor_reviews, blog_reviews, address, business_hours, phone_number, "", description, "", "", ""]
        pass

    while True:
        try:
            keyword_more_link = driver.find_element(By.CLASS_NAME, 'Tvx37')
            keyword_more_link.click()
        except Exception as ex:
            break

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    keyword_number = int(driver.find_element(By.CSS_SELECTOR, "div._Wmab > em").text.replace(",", ""))

    keywords = {}
    ui_tags = soup.find('ui', class_='uNsI9')
    li_tags = soup.find_all('li', class_='nbD78')

    for li in li_tags:
        # 첫 번째 숫자를 int 형으로 변환
        text = li.find('span', class_='nWiXa').text.strip('"')
        numbers = re.findall(r'\d+', li.find('span', class_='TwM9q').text)
        number = int(numbers[0]) if numbers else None
        keywords[text] = number
    
    return [cafe_name, cafe_type, starring, visitor_reviews, blog_reviews, address, business_hours, phone_number, "", description, "", keywords, keyword_number]

In [ ]:
# # WebDriver 초기화
# driver = webdriver.Chrome() # Chrome 드라이버 사용

# for cafe_name in cafe_name_list:
#     # 검색
#     driver.get("https://map.naver.com/p/search/" + cafe_name)
#     time.sleep(3) # 페이지 로드를 위해 잠시 대기
    
#     current_url = driver.current_url
#     url_point = current_url.split("/")

#     if len(url_point) <= 6:
#         # searchIframe으로 컨텍스트 전환 + div scroll 내리기
#         driver.switch_to.frame("searchIframe")
#         scroll_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container")))
#         scroll_down(driver)

#         # 목록을 담고 있는 ul 태그를 찾고, 그 안의 모든 li 태그를 가져옴
#         cafe_list = scroll_div.find_elements(By.TAG_NAME, "li") 

#         # 각 li 태그를 순회하면서 카페이름에 해당하는 a 링크를 클릭
#         for item in cafe_list:
#             try:
#                 cafe_link = item.find_element(By.TAG_NAME, "a")
#                 cafe_link.click()

#                 # 필요한 작업 수행 (예: 상세 정보 수집)
#                 driver.switch_to.default_content()
#                 driver.switch_to.frame("entryIframe")
                
#                 cafe_row = cafe_crawling2()
#                 if cafe_row:
#                     df.loc[len(df)] = cafe_row
#                     print(cafe_row)

#                 # 다음 카페로 이동하기 전에 searchIframe으로 다시 컨텍스트 전환
#                 driver.switch_to.default_content()
#                 driver.switch_to.frame("searchIframe")
#             except Exception as e:
#                 print(f"An error occurred: {e}")
#     else:
#         driver.switch_to.frame("entryIframe")
#         cafe_row = cafe_crawling2()
#         if cafe_row:
#             df.loc[len(df)] = cafe_row
#             print(cafe_row)
#         driver.switch_to.default_content()
#         driver.switch_to.frame("searchIframe")

    
